# Census Data Download Script
This script is for downloading the specified file directories from the US Census Geo-Tiger FTP Site. Please note that this script must be run in Anaconda's Jupyter Lab, and not the ArcPro Jupyter Lab in order to successfully import all modules.



In [2]:
import requests, zipfile, io, csv, os
from bs4 import BeautifulSoup
from zipfile import BadZipfile


# Where you want the files to download to (will create new folders within this workspace) 
#working_directory = "C:/Users/and04671/Documents/Data_Files"
theShapeFileDirectory = "C:/Users/Cole/Documents/SDF"
# The census website. NOTE: this URL will have to be updated to '.../TIGER2019/' and so on, each year
theUrl = "https://www2.census.gov/geo/tiger/TIGER2018/"

# Census download site folder names below
#NOTE: if the folder names change on the census download directory for a new year, change the names in this list
'''
folders = ['AIANNH/','AITSN/','ANRC/','BG/','CONCITY/','COUSUB','CNECTA/','CBSA/','CD/',
           'COUNTY/','CSA/','CSA/','METDIV/','NECTA/','NECTADIV/',
           'PLACE/','PUMA/','ELSD/','SCSD/','UNSD/','STATE/','SLDL/','SLDU/',
           'SUBMCD/','TBG/','TRACT/','TTRACT/','UAC/','ZCTA5/', 
            
           #New for 2018 
           'AREAWATER/','LINEARWATER/','COASTLINE/','FACES/',
           'FACESAH/','FACESAL/','FACESMIL/','FEATNAMES/','PRIMARYROADS/',
           'PRISECROADS/','RAILS/','ROADS/']
'''
#NOTE: original folder pulls did not have Division, Nation, Trust, Res Only, Region as these could not be found...
#on the census site
folders = ['AIANNH/','CONCITY/','STATE/','METDIV/']
for folder in folders:
    fullUrl = theUrl + folder
    r = requests.get(fullUrl)
    webpage = BeautifulSoup(r.text, "html.parser")
    table = webpage.find("table")
    # Find all table row (tr rows)
    tr = table.find_all("tr")
    #print(tr)
    hrefs = []
    # Skipping headers
    for each_tr in tr[3:]:
        td = each_tr.find_all('td')
        # In each tr row find each td cell
        for each_td in td:
            #print(each_td.text) # If you want to check that you are downloading the correct census shapefiles
            if each_td.find('a'): hrefs.append(each_td.find('a')['href'])
    print("Downloading and Extracting files")
    
    
    for c, h in enumerate(hrefs):
        print("Downloading %s of %s" % (c+1, len(hrefs)) )
        urlZip = "%s/%s" % (fullUrl, h)
        theZip = requests.get(urlZip)
        try:
            z = zipfile.ZipFile(io.BytesIO(theZip.content))
            # Creates a file folder to put each type of data into, inside of theShapefileDirectory specified above
            if not os.path.exists(os.path.join(theShapeFileDirectory,folder[:-1])):
                os.makedirs(os.path.join(theShapeFileDirectory,folder[:-1]))
                destination = (os.path.join(theShapeFileDirectory,folder[:-1]))
            # if the file folder with that name already exists, it won't duplicate
            else:
                destination = (os.path.join(theShapeFileDirectory,folder[:-1]))
            z.extractall(destination)
        except BadZipfile:
            pass
